In [1]:
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data=pd.read_csv('yelp_review_processed.csv')

In [5]:
data.head()

,text,label
0,Tried to give this place a second chance and w...,0
1,My Mom ordered penne pasta and received taglia...,0
2,The facility is clean and level however the st...,0
3,Absolutely do not bother There is a coffee bar...,0
4,Inga is the only competent employee here The e...,0


In [6]:
data['label'].value_counts()

0    16667
1    16667
2    16667
Name: label, dtype: int64

In [7]:
data = data.sample(frac=1).reset_index(drop=True) #change order

In [8]:
data = data[data['label'] != 2]#removing neutral

In [9]:
data.head()

,text,label
2,Friends and I stopped in for a quick dinner an...,1
3,The Tour De Malls De Edmonton continues And fa...,0
5,This place is the bomb as you can see from the...,1
6,STAY AWAY THESE GUYS ARE CROOKS They raised t...,0
7,I love this store shabby chic Plus much more c...,1


In [10]:
data.shape

(33334, 2)

In [11]:
train=data
test=pd.read_excel('cz4045data.xlsx')
test=test.iloc[:1000]
test.head()

,uid,restaurant name,rating,content,Label #1,Label #2,Label #3,Label Eval
0,ChdDSUhNMG9nS0VJQ0FnSUNpOW9uQW53RRAB,Konomi Zen,3,Crunchy tempura esp the vegetables,2.0,2.0,2.0,2.0
1,ChdDSUhNMG9nS0VJQ0FnSURJMk5UZXJRRRAB,Vincent Western Food,5,this is one of the best western food i've eate...,1.0,1.0,1.0,1.0
2,ChZDSUhNMG9nS0VJQ0FnSURPazRQdEdREAE,Siam Square Mookata - Best Mookata Restaurant ...,5,Many choice of food to select. Love their teri...,1.0,1.0,1.0,1.0
3,ChdDSUhNMG9nS0VJQ0FnSUMwaTlycm5nRRAB,Old Chang Kee,1,Buying snacks for customers but system mainten...,0.0,0.0,0.0,0.0
4,ChdDSUhNMG9nS0VJQ0FnSUNvdjc2X3BBRRAB,Hiang Ji Cantonese Roasts,1,Seriously overprice and rude service. Avoid at...,0.0,0.0,0.0,0.0


In [12]:
test=pd.DataFrame(test[['content','Label Eval']])
test=test.rename(columns={'content': 'text', 'Label Eval': 'label'})
test = test[test['label'] != 2]#removing neutral
test

,text,label
1,this is one of the best western food i've eate...,1.0
2,Many choice of food to select. Love their teri...,1.0
3,Buying snacks for customers but system mainten...,0.0
4,Seriously overprice and rude service. Avoid at...,0.0
5,The person say closing soon at 730 pm but it c...,0.0
...,...,...
995,Mutton Nasi Biryani\n\n🍜 Food wise: Overall it...,1.0
996,Really nice Zi Char place and pretty affordabl...,1.0
997,"honestly the best mala i’ve had in singapore, ...",1.0
998,"Difficult to get to, food quality is good, the...",1.0


In [13]:
#this part doesn't work for me. But results should be better if emojis and non English are removed
import nltk
from nltk.corpus import words
words = set(words.words('english'))
#!pip install clean-text
from cleantext import clean

LookupError: 
**********************************************************************
  Resource [93mwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('words')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/words[0m

  Searched in:
    - '/Users/tanya.banerjee/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [14]:
for index, row in test.iterrows():
    # removes all emoji
    row['text'] = clean(row['text'],no_emoji=True)
    # removes non english words
    row['text'] = " ".join(w for w in nltk.wordpunct_tokenize(row['text']) if w.lower() in words.words() or not w.isalpha())

test   

NameError: name 'clean' is not defined

In [15]:
text_transformer=TfidfVectorizer(max_features=150000)

In [16]:
#changing train and test data into tfidf
train_text = text_transformer.fit_transform(train['text'])
test_text = text_transformer.transform(test['text'])

In [17]:
train_text.shape, test_text.shape

((33334, 58360), (940, 58360))

In [18]:
#This can be used if we have 3 classes


#logReg=LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
#skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=17)

In [19]:
logReg=LogisticRegression(C=5e1, solver='lbfgs', random_state=17, n_jobs=4)

In [20]:
#cv_results = cross_val_score(logReg, train_text, train['label'], cv=skf, scoring='f1_micro')
#cv_results, cv_results.mean()

In [22]:
logReg.fit(train_text, train['label'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=50.0, n_jobs=4, random_state=17)

In [23]:
test_preds = logReg.predict(test_text)

In [24]:
type(test_preds), test_preds.shape

(numpy.ndarray, (940,))

In [25]:
pd.DataFrame(test_preds, columns=['label'])

,label
0,1
1,1
2,0
3,0
4,0
...,...
935,0
936,1
937,1
938,0


In [26]:
test

,text,label
1,this is one of the best western food i've eate...,1.0
2,Many choice of food to select. Love their teri...,1.0
3,Buying snacks for customers but system mainten...,0.0
4,Seriously overprice and rude service. Avoid at...,0.0
5,The person say closing soon at 730 pm but it c...,0.0
...,...,...
995,Mutton Nasi Biryani\n\n🍜 Food wise: Overall it...,1.0
996,Really nice Zi Char place and pretty affordabl...,1.0
997,"honestly the best mala i’ve had in singapore, ...",1.0
998,"Difficult to get to, food quality is good, the...",1.0


In [27]:
from sklearn.metrics import accuracy_score
score = accuracy_score(test['label'],test_preds)
print(score)

0.8563829787234043


In [ ]:
#I was trying svm but this takes very long to run and doesn't work for some reason
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(train_text, train['label'])
test_preds2=clf.predict(test_text)
score2 = accuracy_score(test['label'],test_preds2)
print(score2)